In [3]:
import MetaTrader5 as mt5
import pandas as pd
%matplotlib qt


In [4]:
# Copying data to pandas data frame
n_days = 365
n_hours = 24
n_mins = 60
aq_window  = n_days * n_hours * n_mins
plot_window = 72

In [14]:
# Initializing MT5 connection 
mt5.initialize()


print(mt5.terminal_info())
print(mt5.version())
stockdata = pd.DataFrame()
rates = mt5.copy_rates_from_pos("EURUSD", mt5.TIMEFRAME_H1,0,100)

#rates = np.flip(rates,0)
rates.shape

TerminalInfo(community_account=True, community_connection=True, connected=True, dlls_allowed=False, trade_allowed=False, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=False, build=2755, maxbars=100000, codepage=0, ping_last=65325, community_balance=0.0, retransmission=0.0, company='MetaQuotes Software Corp.', name='MetaTrader 5', language='English', path='C:\\Program Files\\MetaTrader 5', data_path='C:\\Users\\hamza\\AppData\\Roaming\\MetaQuotes\\Terminal\\D0E8209F77C8CF37AD8BF550E51FF075', commondata_path='C:\\Users\\hamza\\AppData\\Roaming\\MetaQuotes\\Terminal\\Common')
(500, 2755, '15 Jan 2021')


(100,)

In [17]:
data_frame = pd.DataFrame(rates,columns=['time','open','high','low','close','nn','nn1','nn2']).drop(['nn','nn1','nn2'],axis=1)

In [24]:
data_frame['date'] = pd.Timestamp.to_pydatetime(data_frame['time'])

TypeError: descriptor 'to_pydatetime' for 'pandas._libs.tslibs.timestamps._Timestamp' objects doesn't apply to a 'Series' object

In [26]:
data_frame.tail()

,time,open,high,low,close,date
95,1614016800,1.21389,1.21562,1.21354,1.21547,1970-01-01 00:00:01.614016800
96,1614020400,1.21547,1.21677,1.21532,1.21654,1970-01-01 00:00:01.614020400
97,1614024000,1.21654,1.21694,1.21634,1.21655,1970-01-01 00:00:01.614024000
98,1614027600,1.21656,1.21691,1.21647,1.21674,1970-01-01 00:00:01.614027600
99,1614031200,1.21674,1.21674,1.21619,1.21619,1970-01-01 00:00:01.614031200


In [7]:
data_frame.describe()

,time,open,high,low,close
count,2.400000e+03,2400.000000,2400.000000,2400.000000,2400.000000
mean,1.601018e+09,1.179116,1.179879,1.178375,1.179148
std,3.494731e+06,0.011264,0.011248,0.011276,0.011259
min,1.594926e+09,1.137860,1.138340,1.137030,1.137890
25%,1.597949e+09,1.174100,1.174690,1.173360,1.174127
50%,1.600972e+09,1.180930,1.181585,1.180170,1.180940
75%,1.603995e+09,1.185242,1.185963,1.184505,1.185260
max,1.607022e+09,1.216120,1.217460,1.215700,1.216130


In [13]:
# prepare the buy request structure
symbol = "XAUUSD"
symbol_info = mt5.symbol_info(symbol)
if symbol_info is None:
    print(symbol, "not found, can not call order_check()")
    mt5.shutdown()
    quit()
 
# if the symbol is unavailable in MarketWatch, add it
if not symbol_info.visible:
    print(symbol, "is not visible, trying to switch on")
    if not mt5.symbol_select(symbol,True):
        print("symbol_select({}}) failed, exit",symbol)
        mt5.shutdown()
        quit()
 
lot = 0.1
point = mt5.symbol_info(symbol).point
price = mt5.symbol_info_tick(symbol).ask
deviation = 20
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_BUY,
    "price": price,
    "sl": price - 100 * point,
    "tp": price + 100 * point,
    "deviation": deviation,
    "magic": 234000,
    "comment": "python script open",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_RETURN,
}
 
# send a trading request
result = mt5.order_send(request)
# check the execution result
print("1. order_send(): by {} {} lots at {} with deviation={} points".format(symbol,lot,price,deviation));
if result.retcode != mt5.TRADE_RETCODE_DONE:
    print("2. order_send failed, retcode={}".format(result.retcode))
    # request the result as a dictionary and display it element by element
    result_dict=result._asdict()
    for field in result_dict.keys():
        print("   {}={}".format(field,result_dict[field]))
        # if this is a trading request structure, display it element by element as well
        if field=="request":
            traderequest_dict=result_dict[field]._asdict()
            for tradereq_filed in traderequest_dict:
                print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
    print("shutdown() and quit")
    mt5.shutdown()


1. order_send(): by XAUUSD 0.1 lots at 1837.62 with deviation=20 points


In [ ]:
import time
while(True):
    print(mt5.symbol_info_tick(symbol))
    time.sleep(0.5)

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

series=list()
window_size = 3
forecast = []
for time in range(len(series) - window_size):
    forecast.append(model.predict(series[time:time + window_size][np.newaxis]))